In [2]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph
from langchain.chat_models import ChatOpenAI
from langchain_gigachat.chat_models import GigaChat
from langchain.schema import HumanMessage, SystemMessage, Document, AIMessage
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.types import Command
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState, START
from typing import TypedDict, Optional, Literal, List, Dict
from typing_extensions import TypedDict

import datetime
import warnings
import os
import re

In [3]:
warnings.filterwarnings("ignore")

In [4]:
os.environ["GIGACHAT_CREDENTIALS"] = "Yjc1YWZhNTItMzYwYS00NmU4LTk4YjctZjU4YzAwMDIyMGJmOjJhNWE3YmVhLTAyYzctNGJhNy05NWE3LWEzY2YwNGQzYzZiNw=="

In [59]:
with open("../sources/ba_instruction.md", "r") as f:
    ba_instruction = f.read()

def save_result(result):
    format_ = "%Y_%m_%dT%H_%M_%S"
    datetime_mark = datetime.datetime.now().strftime(format_)
    
    with open(f"../tmp/ba_{datetime_mark}.md", "w") as f:
        f.write(result["result"])

In [5]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False,
    streaming=False,
    max_tokens=8192,
    temperature=0.3,
    repetition_penalty=1.01
)

In [6]:
tools = []

In [7]:
ba_agent = create_react_agent(llm, tools=[], checkpointer=MemorySaver())

In [8]:
class AgentState(TypedDict):
    task: str
    messages: List
    result: str
    
class ValidatorResponse(TypedDict):
    """В поле instructions нужно указать рекомендации. В поле status указать статус задачи [FAIL, SUCCESS]"""
    instructions: str
    status: Literal["FAIL", "SUCCESS"]

In [32]:
def ba_agent_node(state:AgentState)->Command[Literal["ba_validator"]]:    
    print("Status: ba_agent_node")
    if "messages" in state and state["messages"]:
        old_messages = state["messages"]
        request = state["messages"][-1].content
    else:
        old_messages = []
        request = ba_prompt.format(task=state["task"], ba_instruction=ba_instruction)
    request = {
            "messages": [HumanMessage(content=request)]
        }
    response = ba_agent.invoke(request, config=config)
    if isinstance(response, dict):
        result = response["messages"][-1].content
    else:
        result = response
    return Command(
        update={
            "result": result,
            "messages": old_messages + [HumanMessage(content=result, name="Аналитик")]
        },
        goto="ba_validator"
    )

def ba_validator_node(state:AgentState)->Command[Literal["ba_agent", END]]:
    print("Status: ba_validator_node")
    return_node = "ba_agent"
    if not "messages" in state or not state["messages"]:
        return Command(goto=return_node)
    
    prompt = ba_validator_prompt.format(task=state["task"], format=format_)
    system = SystemMessage(content=prompt)
    request = [system] + state["messages"]

    validator_msgs_fount = len(list(filter(lambda x: x.name=="Валидатор", state["messages"])))
    response = llm.invoke(request)
    
    response = response.content
    print(f"RESPONSE: {response}")
    match_json = re.search(r'```json\s*(.*?)\s*```', response, re.DOTALL)
    if match_json:
        response = eval(match_json.group(1))
    else:
        response = {"status":"SUCCESS", "instructions":""}
    
    result = response["instructions"]
    print(f"STATUS: {response['status']}\n\nVALIDATOR: {result}")
    
    if response["status"]=="SUCCESS" or validator_msgs_fount+1 > 3:
        goto = END
    else:
        goto = return_node
    return Command(
            update={
            "messages": state["messages"] + [HumanMessage(content=result, name="Валидатор")],
            },
            goto=goto
        )

In [33]:
ba_prompt = (
    "Ты - превосходный бизнес-аналитик."
    " Твоя задача - составить бизнес-требования к задаче {task} в соответствии с инструкцией {ba_instruction}."
)

In [34]:
format_ ="""{
    "instructions": "Здесь укажи свои рекомендации или оставь строку пустой, если рекомендаций нет",
    "status": "FAIL" если требуется доработка иначе "SUCCESS"
}"""

In [35]:
ba_validator_prompt = """Ты - Валидатор. Ты прекрасно умеешь критиковать написанные бизнес-требования для достижения наилучшего результата. 
Ты работаешь вместе с Аналитиком. Проанализуруй бизнес-требования, которые предложил аналитик к задаче {task}. 
- Проверь, что бизнес-требования, которые предложил аналитик, хорошо подходят для решения задачи.
- Проверь, что хорошо описаны как основные сценарии, так и альтернативные.
- Проверь, что хорошо явно выделена User Story.

Если сценарий нужно улучшать, напиши свои замечания и верни сценарий на дорабоку, ответив FAIL В конце инструкции попроси аналитика заново полностью описать требования, а не вносить изменения в существующие. 
Если Аналитик хорошо выполнил свою работу и замечаний нет, ответь SUCCESS.

Ответ дай в следующем формате json:
```json
{format}
```
"""

In [36]:
config = {"configurable": {"thread_id": "thread_id123"}}

In [37]:
state_schema = AgentState
builder = StateGraph(state_schema=state_schema)
builder.set_entry_point("ba_agent")
builder.add_node("ba_agent", ba_agent_node)
builder.add_node("ba_validator", ba_validator_node)
graph = builder.compile()

In [38]:
with open("../tmp/desc_2025_05_18T03_55_01.md", "r") as f:
    task = f.read() 

In [39]:
initial_state = {
    "task": task
}

In [40]:
result = graph.invoke(initial_state)

Status: ba_agent_node
Status: ba_validator_node
RESPONSE: ```json
{
    "instructions": "В целом, бизнес-требования хорошо структурированы и охватывают все необходимые аспекты проекта. Однако, есть несколько моментов, которые стоит уточнить или улучшить для достижения наилучшего результата:\n\n1. **User Story явно не выделена**: В требованиях не указана конкретная User Story, которая бы четко описывала, кто, что и зачем делает. Это важно для понимания конечной цели проекта и его ценности для пользователей.\n\n2. **Детализация взаимодействия между агентами**: В разделе 'Взаимодействие между агентами' можно добавить больше деталей о том, как именно происходит передача данных между агентами, какие форматы данных используются и как обеспечивается безопасность передачи данных.\n\n3. **Обработка ошибок**: В разделе 'Альтернативные сценарии' можно добавить больше информации о том, как именно обрабатываются ошибки, какие действия предпринимаются в случае возникновения ошибок и как система увед

In [60]:
save_result(result)